In [32]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.EnumerateStereoisomers import EnumerateStereoisomers, StereoEnumerationOptions
import py3Dmol

In [34]:
def chiral_check(smiles):
    """
    Checks input molecule for chemically impossible stereochemistry and enforces 
    feasible stereochemistry.
    
    Input:
    smiles: SMILES as a string
    
    Returns:
    SMILES outputed as a string with acceptable stereochemistry
    """
    mol = Chem.AddHs(Chem.MolFromSmiles(smiles))
    #   Create tuple of (atom object, atom number)
    get_atoms = mol.GetAtoms()
    atom_num_list = []
    for index in mol.GetAtoms():
        atom_num_list.append((index, index.GetIdx()))
    #   Collect all chiral centers
    chiral_centers = Chem.FindMolChiralCenters(mol, includeUnassigned=True)
    chiral_atom_dict = {}
    for atoms in range(len(chiral_centers)):
        chiral_atom_dict[chiral_centers[atoms][0]] = []
    #   Create ring object to add ring numbers to corresponding chiral center
    ring_info = mol.GetRingInfo()
    for number in range(ring_info.NumRings()):
        for atoms in chiral_atom_dict:
            if atoms in ring_info.AtomRings()[number]:
              chiral_atom_dict[atoms].append(number)
    #   Create tuples of chiral centers which are required to be restricted
    tmp_i = int()
    tmp_j = int()
    set_chirals = []
    for i in chiral_atom_dict:
        for j in chiral_atom_dict:
            if i < j and chiral_atom_dict[i] == chiral_atom_dict[j]:
                for n in atom_num_list:
                    if n[1] == i:
                        tmp_i = n
                    elif n[1] == j:
                        tmp_j = n
                set_chirals.append((tmp_i, tmp_j))
                
    #    Enforce correct stereochemistry
    if len(set_chirals) > 0:
        for check in set_chirals:
            if check[0][0].GetChiralTag() == check[1][0].GetChiralTag():
                if check[0][0].GetChiralTag() == Chem.CHI_TETRAHEDRAL_CW:
                    check[0][0].SetChiralTag(Chem.CHI_TETRAHEDRAL_CCW)
                elif check[0][0].GetChiralTag() == Chem.CHI_TETRAHEDRAL_CCW:
                    check[0][0].SetChiralTag(Chem.CHI_TETRAHEDRAL_CW)
    filtered_smiles = Chem.MolToSmiles(mol, isomericSmiles=True)
    return filtered_smiles

In [48]:
#    Known problem case for testing
#    AllChem.EmbedMolecule(Chem.AddHs(Chem.MolFromSmiles(fix_chiral))) will stall for molbad (chemically impossible) but not for molgood or mol null
molbad = 'CCSCc1ccc2c(c1)c1c3c(c4c5cc(CSCC)ccc5n5c4c1n2[C@@H]1C[C@H](C(=O)OOC)[C@]5(C)O1)CNC3=O'
molgood = 'CCSCc1ccc2c(c1)c1c3c(c4c5cc(CSCC)ccc5n5c4c1n2[C@@H]1C[C@H](C(=O)OOC)[C@@]5(C)O1)CNC3=O'
molnull = 'CCSCc1ccc2c(c1)c1c3c(c4c5cc(CSCC)ccc5n5c4c1n2[CH]1C[CH](C(=O)OOC)[C]5(C)O1)CNC3=O'

mol = molgood
#    molbad should be only SMILES to give issue if 'chiral_check
mol = chiral_check(mol)
AllChem.EmbedMolecule(Chem.AddHs(Chem.MolFromSmiles(mol)))
print('done')

done


In [ ]:
#    This sample was taken, verbatim, from the MolecularInsights notebook to show how to incorperate the appropriate function
#    Below section should be copied for scaffold generation as well
%%time
test_generated_smiles_linkers = designer.linker_generation(
    *linker_generation,
    #n_samples_per_trial=250,
    n_samples_per_trial=500,
    n_trials=2,
    sanitize=True,
    do_not_fragment_further=False,
    random_seed=100,
)

#    All generated smiles are taken in, checked for chirality issues, fixed where necessary, and checked again to see if EmbedMolecule stalls
generated_smiles_linkers = []
for test in test_generated_smiles_linkers:
  fix_chiral = chiral_check(test)
  generated_smiles_linkers.append(fix_chiral)
  #print(fix_chiral)
  AllChem.EmbedMolecule(Chem.AddHs(Chem.MolFromSmiles(fix_chiral)))

dm.to_image(generated_smiles_linkers[:5], mol_size=(350, 200))